In [2]:
import pyodbc
from meyerDB import cable_connection
import PyQt5
import matplotlib.pyplot as plt
%matplotlib qt
import numpy as np
from IPython.display import HTML, display, clear_output

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeavePOut
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor
import sklearn.preprocessing as pp
from scipy.optimize import curve_fit
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, RBF2, WhiteKernel, ConstantKernel, RationalQuadratic
from scipy.stats import norm

# init db connection
conn = pyodbc.connect(cable_connection)
cursor = conn.cursor()
print('Database connection ok')

def display_table(data):
    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            try:
                value = str(round(100*field, 1)).replace('.', ',') + '%'
            except:
                value = field
            html += "<td><h4>%s</h4><td>"%(value)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

def logifunc(x,x0,k,l, A):
    #l = 2300
    return l / (1 + A*np.exp(-k*(x - x0)))

Database connection ok


In [3]:
# Design progress
# Predictors:
# Time series, GT
# Algorithms:
# Gaussian process regression

def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

def calc_error():

    res = y_mean[xidxs_inv]*gt - y[xidxs_inv]
    err = abs(res).sum()/(y[xidxs_inv]).sum()
    return err
    
def pplot():


    plt.plot(x[xidxs_inv], y[xidxs_inv], 'r')
    plt.plot(xx*gt, yy*gt, 'g')
    plt.plot(x, y_mean*gt, 'b--')
    #plt.ylim([0, 1.3*y[-1]])
    #plt.xlim([-100, 0])
    plt.pause(0.17)
    plt.cla()

# Get the data
cursor.execute("SELECT r.project_id, gross_tonnage, sum(amount) FROM routed as r"
    " LEFT JOIN projects as p ON p.project_id=r.project_id"
    " GROUP BY r.project_id, gross_tonnage")
data = np.array(cursor.fetchall())
ships = data[:, 0]
gts = data[:, 1]
cables = data[:, -1]
# init lpo split
p = 2 #ships
lpo = LeavePOut(p)

#init linear pred
model = linear_model.LinearRegression()

wk = 'rw'
cursor.execute("UPDATE progress SET cables=0")
cursor.execute(

    "UPDATE progress SET progress.cables=t1.cables FROM"
    " (SELECT project_id, {}, sum(amount) as cables FROM ship_readiness GROUP BY project_id, {}) t1"
    " WHERE t1.project_id=progress.project_id AND t1.{}=progress.wk".format(wk, wk, wk)
)
cursor.commit()

# get progress data
ship_data = {}
for ship in ships:
    cursor.execute("SELECT wk, cables FROM progress WHERE project_id={} and cables > 100 ORDER BY wk".format(ship))
    data = cursor.fetchall()
    ship_data[ship] = np.array(data)

# init gaussian process regressor
kernel = ConstantKernel(constant_value=10, constant_value_bounds='fixed') * RBF(
    length_scale=0.0001, length_scale_bounds='fixed')
gp = GaussianProcessRegressor(kernel=kernel,
                            alpha=0.01)


cv_errors = []
for train_i, test_i in lpo.split(ships):
    model.fit(gts[train_i].reshape(-1, 1), cables[train_i])
    errors = []
    completeness = []
    prior_x = []
    prior_y = []
    prior_gt = []
    for i in range(11):
        ship = ships[i]
        gt = gts[i]
        prior_x.append(ship_data[ship][:, 0])
        prior_y.append(ship_data[ship][:, 1].cumsum())
        prior_gt.append(gt*np.ones_like(ship_data[ship][:, 0]))
    prior_x = np.concatenate(prior_x, axis=0)
    prior_y = np.concatenate(prior_y, axis=0)
    prior_gt = np.concatenate(prior_gt, axis=0)


    for i in test_i:
    
        ship = ships[i]
        gt = gts[i]
        linear_pred = model.predict(gts[[i]].reshape(1, 1))[0]

        x = ship_data[ship][:, 0]
        y = ship_data[ship][:, 1].cumsum()
        

        for wk in np.arange(-100, 1):
            try:
                xidxs = np.argwhere(x <= wk)[:, 0]
                if xidxs.shape[0] == 0: continue
                xidxs_inv = np.argwhere(x > wk)[:, 0]

                prior_idxs = np.argwhere(prior_x > wk)[:, 0]
            except IndexError:
                continue
            x_scaled = x/gt
            y_scaled = y/gt
            prior_x_scaled = prior_x/prior_gt
            prior_y_scaled = prior_y/prior_gt
            xx = x_scaled[xidxs]
            yy = y_scaled[xidxs]

            estimated_completeness = yy[-1]/(linear_pred/gt)
            if estimated_completeness < 0.001: continue
            #real_completeness = yy[-1]/goal
            X = np.hstack((xx, prior_x_scaled[prior_idxs]))
            Y = np.hstack((yy, prior_y_scaled[prior_idxs]))
            weight = 5
            for w in range(wheight):
                X = np.hstack((xx, X))
                Y = np.hstack((yy, Y))
            gp.fit(X.reshape(-1, 1), Y.reshape(-1, 1))
            y_mean, y_cov = gp.predict(x_scaled.reshape(-1, 1), return_cov=True)



            clear_output(wait=True)

            err = calc_error()

            errors.append(err)
            #completeness.append(real_completeness)

            pplot()


    errors = np.array(errors)
    cv_errors.append(errors.mean())
cv_errors = np.array(cv_errors)            
print(cv_errors.mean(), cv_errors.std(), cv_errors.max())




NameError: name 'wheight' is not defined